In [1]:
# connect mysql
import pymysql
conn = pymysql.connect(host = 'localhost', user='dongyeon0317', password='1Q2w3e4r!@', db='crime_prediction', charset = 'utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)

In [2]:
# select from Dataset - Choose scenario

# Season
num = int(input('1.Spring 2.Summer 3.Fall 4.Winter '))
if num == 1: season = 'Spring'
elif num == 2: season = "Summer" 
elif num == 3: season = "Fall" 
else: season = "Winter"

# Time    
num = int(input("1.Morning 2.Afternoon 3.Evening 4.Midnight "))
if num == 1: time = "Morning"  
elif num == 2: time = "Afternoon" 
elif num == 3: time = "Evening" 
else: time = "Midnight"
    
# Weather    
num = int(input("1.눈/비 2.맑음 3.흐림 "))
if num == 1: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall > 0) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall > 0);' 
elif num == 2: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud < 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud < 5);' 
else: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud >= 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud >= 5);' 

dataset_sql = 'SELECT grid, CCTV_num, police_value, resident, floating FROM Dataset ' + where
grid_sql = 'SELECT grid, CCTV_num, police_value, resident, floating FROM Grid WHERE time = \''+ time + '\''
curs.execute(dataset_sql)
dataset_result = curs.fetchall()
curs.execute(grid_sql)
grid_result = curs.fetchall()

1.Spring 2.Summer 3.Fall 4.Winter 1
1.Morning 2.Afternoon 3.Evening 4.Midnight 3
1.눈/비 2.맑음 3.흐림 2


In [3]:
# Transform to Pandas Dataframe
import pandas as pd
dataset_result = pd.DataFrame(dataset_result)
grid_result = pd.DataFrame(grid_result)
result = pd.concat([dataset_result, grid_result])
result['crime_num'] = result.groupby(['grid']).grid.transform('count') -1
result = result.drop_duplicates()
result = result.sort_values('grid')
result

,grid,CCTV_num,police_value,resident,floating,crime_num
429,0,0,0,20,930,34
8,1,0,0,30,670,21
889,2,0,0,20,810,22
100,3,0,0,20,660,32
969,4,0,0,40,760,18
...,...,...,...,...,...,...
15888,1102,8,0,310,6430,24
16185,1103,0,0,330,6250,55
16132,1104,0,0,450,6400,43
16140,1105,0,0,240,6770,51


In [4]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz  # DecisionTree 모듈, 시각화 모듈
from sklearn.model_selection import train_test_split  # train, test dataset 분류
from sklearn.model_selection import GridSearchCV # cross validation, hyper parameter
from collections import Counter # 라벨 분포 확인
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import sys
import pydot
np.set_printoptions(threshold=sys.maxsize)
# Decision tree Classifier 생성

# 학습해야하는 변수
data = result[["CCTV_num", "police_value", "resident", "floating"]].to_numpy()
# 맞춰야하는 변수: crime_num
target = result.crime_num.to_numpy()

# dataset 분할
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=10)

# 하이퍼 파라미터 찾기
tree = DecisionTreeRegressor()
parameters = {'criterion':['mse'], 'max_depth' : [x for x in range(3, 31, 1)], 'min_samples_leaf' : [x for x in range(1, 30, 1)], 'min_samples_split' : [x for x in range(2, 30, 1)]}

# param_grid의  하이퍼 파라미터를 5개의 train, test set fold로 나누어 테스트 수행 설정
## refit=True가 기본 설정으로 True이면 가장 좋은 파라미터 설정으로 재학습 시킴
grid_dtree = GridSearchCV(tree, param_grid=parameters, cv=5, refit=True)

#학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습, 평가
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과를 추출해 데이터 프레임으로 반환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'criterion': 'mse', 'max_depth': 3, 'min_samp...",0.808660,2977,0.849662,0.803897,0.849514
1,"{'criterion': 'mse', 'max_depth': 3, 'min_samp...",0.808660,2977,0.849662,0.803897,0.849514
2,"{'criterion': 'mse', 'max_depth': 3, 'min_samp...",0.808660,2977,0.849662,0.803897,0.849514
3,"{'criterion': 'mse', 'max_depth': 3, 'min_samp...",0.808660,2977,0.849662,0.803897,0.849514
4,"{'criterion': 'mse', 'max_depth': 3, 'min_samp...",0.808660,2977,0.849662,0.803897,0.849514
...,...,...,...,...,...,...
22731,"{'criterion': 'mse', 'max_depth': 30, 'min_sam...",0.710912,16832,0.728144,0.682177,0.702083
22732,"{'criterion': 'mse', 'max_depth': 30, 'min_sam...",0.710912,16832,0.728144,0.682177,0.702083
22733,"{'criterion': 'mse', 'max_depth': 30, 'min_sam...",0.710912,16832,0.728144,0.682177,0.702083
22734,"{'criterion': 'mse', 'max_depth': 30, 'min_sam...",0.710912,16832,0.728144,0.682177,0.702083


In [5]:

print('GridSearch 최적 파라미터: ', grid_dtree.best_params_)
print('GridSearch 최고 점수: ', grid_dtree.best_score_)

# GridSearchCV의 refit으로 학습된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estmator_ 는 이미 최적 학습이 됐으므로 별도 학습이 필요 없음
pred = estimator.predict(X_test)

# predic test_set labels
y_pred = estimator.predict(data)
# Compute test-set RMSE
mse_dt = MSE(target, y_pred)
rmse_dt = mse_dt**(1/2)
# Print rmse_dt
print("Grid RMSE: ", rmse_dt, '\n')
print(y_pred)

# 시각화를 위해 학습에 사용한 feature 라벨 추출
feature_names = result.columns.tolist()
feature_names = feature_names[1:5]
# 트리 시각화
dot_data = export_graphviz(estimator, out_file="1_1_1_DecisionTree.dot", feature_names = feature_names, filled=True, rounded=True, special_characters=True)
# 인코딩
(graph,) = pydot.graph_from_dot_file("1_1_1_DecisionTree.dot", encoding='utf8')
# png로 저장
graph.write_png("1_1_1_DecisionRegress.png") 

# 전체 Grid의 범죄 발생 확률 구하기
total = np.sum(y_pred)
result_prob = y_pred / total
#print(result_prob)


GridSearch 최적 파라미터:  {'criterion': 'mse', 'max_depth': 4, 'min_samples_leaf': 3, 'min_samples_split': 17}
GridSearch 최고 점수:  0.8288432803018647
Grid RMSE:  4.81290493362683 

[21.64137931 21.64137931 21.64137931 21.64137931 21.64137931 21.64137931
 21.64137931 10.38297872 10.38297872 21.64137931 21.64137931 21.64137931
 21.64137931 10.38297872 21.64137931 21.64137931 21.64137931 10.38297872
 10.38297872 21.64137931 21.64137931 21.64137931 21.64137931 21.64137931
 21.64137931 21.64137931 10.75       10.75       21.64137931 10.75
  8.63157895 21.64137931 10.38297872 21.64137931 21.64137931 21.64137931
 10.75       10.75       10.75       10.75       10.75       21.64137931
 21.64137931 21.64137931 21.64137931 21.64137931 21.64137931 21.64137931
 10.75        8.63157895 10.75       10.75        8.63157895 21.64137931
 21.64137931 21.64137931 21.64137931 21.64137931 10.38297872 21.64137931
 21.64137931 21.64137931 21.64137931 10.38297872 10.75        8.63157895
 10.75       10.75       10.

In [ ]:
###################################################################################################################################

In [ ]:
#DecisionTreeRegressor - 최적 depth 뽑기
from sklearn.tree import DecisionTreeRegressor, export_graphviz  # DecisionTree 모듈, 시각화 모듈
from sklearn.model_selection import train_test_split  # train, test dataset 분류
from collections import Counter # 라벨 분포 확인
from sklearn.metrics import mean_squared_error as MSE
import matplotlib.pyplot as plt
import numpy as np
import sys
from tqdm import tqdm_notebook
np.set_printoptions(threshold=sys.maxsize)

# 학습해야하는 변수
data = result[["CCTV_num", "police_value", "resident", "floating"]].to_numpy()
# 맞춰야하는 변수: crime_num
target = result.crime_num.to_numpy()

# dataset 분할
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=10)

# 데이터 분할 상태 확인
print('train set: ', Counter(y_train))
print('test set: ', Counter(y_test), '\n')

train_arr = []
test_arr = []
grid_arr = []
RMSE_arr = []

for k in tqdm_notebook(range(1, 21)):
    
    # X_train, Y_train 입력해 모델 생성, 학습 - Gini 계수
    tree = DecisionTreeRegressor(random_state=42, max_depth=k, criterion='mse', splitter='best')
    tree = tree.fit(X_train, y_train)
    
    # 정확도 측정 - RMSE 측정
    y_pred = tree.predict(data)
    mse_dt = MSE(target, y_pred)
    rmse_dt = mse_dt**(1/2)
    RMSE_arr.append(rmse_dt)
    
    # train set 정확도 확인
    Y_pred = np.around(tree.predict(X_train))
    train_accuracy = 0
    for i in range(len(X_train)):
        if Y_pred[i] == y_train[i]:
            train_accuracy += 1
    train_accuracy = round(train_accuracy / len(X_train),3)
    train_arr.append(train_accuracy)
    
    # test set 정확도 확인
    Y_pred = np.around(tree.predict(X_test))
    test_accuracy = 0
    for i in range(len(X_test)):
        if Y_pred[i] == y_test[i]:
            test_accuracy += 1
    test_accuracy = round(test_accuracy / len(X_test),3)
    test_arr.append(test_accuracy)

plt.figure(figsize=(15, 8))
plt.plot(train_arr, label='train', color='green')
plt.plot(test_arr, label='test', color='blue')
plt.plot(RMSE_arr, label='RMSE', color='red')
plt.legend()
plt.xlabel('Depth')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# Decision tree Regressor 생성

import pydot
# decision tree Regresssor 생성
tree = DecisionTreeRegressor(random_state = 42, criterion = 'mse', splitter = 'best', max_depth = 4)
# 모델 생성
tree.fit(X_train, y_train)

# predic test_set labels
y_pred = tree.predict(data)
# Compute test-set RMS
mse_dt = MSE(target, y_pred)
rmse_dt = mse_dt**(1/2)
# Print rmse_dt
print("Grid RMSE: ", rmse_dt, '\n')

# 정확도 확인
Y_pred = np.around(tree.predict(X_train))
test_accuracy = 0
for i in range(len(X_train)):
    if Y_pred[i] == y_train[i]:
        test_accuracy += 1
test_accuracy = round(test_accuracy / len(X_train),3)
print("내가 만든거: ", test_accuracy)
#print("Train Set Accuracy: {:.3f}".format(tree.score(X_train, y_train)))

Y_pred = np.around(tree.predict(X_test))
test_accuracy = 0
for i in range(len(X_test)):
    if Y_pred[i] == y_test[i]:
        test_accuracy += 1
test_accuracy = round(test_accuracy / len(X_test),3)
print("내가 만든거: ", test_accuracy)
#print("Test Set Accuracy: {:.3f}".format(tree.score(X_test, y_test)))

# 시각화를 위해 학습에 사용한 feature 라벨 추출
feature_names = result.columns.tolist()
feature_names = feature_names[1:5]
# 트리 시각화
dot_data = export_graphviz(tree, out_file="1_1_1_DecisionTree.dot", feature_names = feature_names, filled=True, rounded=True, special_characters=True)
# 인코딩
(graph,) = pydot.graph_from_dot_file("1_1_1_DecisionTree.dot", encoding='utf8')
# png로 저장
graph.write_png("1_1_1_DecisionTreeRegress.png") 

# 전체 Grid의 범죄 발생 확률 구하기
total = np.sum(y_pred)
result_prob = y_pred / total
#print(result_prob)

In [ ]:
#DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor, export_graphviz # DecisionTree 회귀 모델, 시각화 모듈
from sklearn.model_selection import train_test_split # train, test dataset 분류
from sklearn.metrics import mean_squared_error as MSE
import pydot
import numpy as np
import math
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)

# 학습해야하는 변수
data = result[["CCTV_num", "police_value", "resident", "floating"]].to_numpy()
# 맞춰야하는 변수: crime_num
target = result.crime_num.to_numpy()

# dataset 분할
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=3)

# 데이터 분할 상태 확인
print('train set: ', Counter(y_train))
print('test set: ', Counter(y_test), '\n')

# decision tree Regresssor 생성
tree = DecisionTreeRegressor(random_state = 3, criterion = 'mse', splitter = 'best')
# 모델 생성
tree.fit(X_train, y_train)

# predic test_set labels
y_pred = tree.predict(data)
# Compute test-set RMS
mse_dt = MSE(target, y_pred)
rmse_dt = mse_dt**(1/2)
# Print rmse_dt
print("Grid RMSE: ", rmse_dt, '\n')

# 정확도 확인
Y_pred = np.around(tree.predict(X_train))
test_accuracy = 0
for i in range(len(X_train)):
    if Y_pred[i] == y_train[i]:
        test_accuracy += 1
test_accuracy = round(test_accuracy / len(X_train),3)
print("내가 만든거: ", test_accuracy)
print("Train Set Accuracy: {:.3f}".format(tree.score(X_train, y_train)))

Y_pred = np.around(tree.predict(X_test))
test_accuracy = 0
for i in range(len(X_test)):
    if Y_pred[i] == y_test[i]:
        test_accuracy += 1
test_accuracy = round(test_accuracy / len(X_test),3)
print("내가 만든거: ", test_accuracy)
print("Test Set Accuracy: {:.3f}".format(tree.score(X_test, y_test)))

# 시각화를 위해 학습에 사용한 feature 라벨 추출
feature_names = result.columns.tolist()
feature_names = feature_names[1:5]
# 트리 시각화
dot_data = export_graphviz(tree, out_file="1_1_1_DecisionTree.dot", feature_names = feature_names, filled=True, rounded=True, special_characters=True)
# 인코딩
(graph,) = pydot.graph_from_dot_file("1_1_1_DecisionTree.dot", encoding='utf8')
# png로 저장
graph.write_png("1_1_1_DecisionTreeRegress.png") 

# 전체 Grid의 범죄 발생 확률 구하기
total = np.sum(y_pred)
result_prob = y_pred / total
print(result_prob)


# 그래프 시각화
'''
plt.figure()
plt.plot(X_test, y_pred, color = 'cornflowerblue', label = 'test')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Decision Tree Regression')
plt.legend()
plt.show()
'''

In [ ]:
################################ CCTV 수, 경찰서, 유동인구, 거주인구 Grid 번호 뽑기 ################################
import pandas as pd
import numpy as np

# select from Dataset
sql = 'SELECT * FROM Grid;'
curs.execute(sql)
result = curs.fetchall()

# Transform to Pandas Dataframe
result = pd.DataFrame(result)

# 필요한 데이터프레임만 뽑아오기
total = result[["grid", "time", "CCTV_num", "police_value", "resident", "floating"]]
total = total.drop_duplicates(["grid", "time"])
total = total.sort_values(by=['grid'], axis=0)

# 평균값 구하기
CCTV_num = result["CCTV_num"]
CCTV_num_mean = np.mean(CCTV_num).round()

police_value = result["police_value"]
police_value_mean = np.mean(police_value).round()

resident = result["resident"]
resident_mean = np.mean(resident).round()

# 시간대별로 나누기
morning_df = total.loc[total["time"] == "Morning"]
afternoon_df = total.loc[total["time"] == "Afternoon"]
evening_df = total.loc[total["time"] == "Evening"]
midnight_df = total.loc[total["time"] == "Midnight"]

# 시간대별 유동인구 평균값 구하기
floating_mean = [np.mean(morning_df["floating"]).round(), np.mean(afternoon_df["floating"]).round(), np.mean(evening_df["floating"]).round(), np.mean(midnight_df["floating"]).round()]

# 조건에 모두 만족하는 그리드 번호 추출
grid_all = []
morning_grid_all = (morning_df[(morning_df["CCTV_num"] <= CCTV_num_mean) & (morning_df["police_value"] <= police_value_mean) & (morning_df["resident"] >= resident_mean) & (morning_df["floating"] >= floating_mean[0])])["grid"].values
grid_all.append(morning_grid_all)

afternoon_grid_all = (afternoon_df[(afternoon_df["CCTV_num"] <= CCTV_num_mean) & (afternoon_df["police_value"] <= police_value_mean) & (afternoon_df["resident"] >= resident_mean) & (afternoon_df["floating"] >= floating_mean[1])])["grid"].values
grid_all.append(afternoon_grid_all)

evening_grid_all = (evening_df[(evening_df["CCTV_num"] <= CCTV_num_mean) & (evening_df["police_value"] <= police_value_mean) & (evening_df["resident"] >= resident_mean) & (evening_df["floating"] >= floating_mean[2])])["grid"].values
grid_all.append(evening_grid_all)

midnight_grid_all = (midnight_df[(midnight_df["CCTV_num"] <= CCTV_num_mean) & (midnight_df["police_value"] <= police_value_mean) & (midnight_df["resident"] >= resident_mean) & (midnight_df["floating"] >= floating_mean[3])])["grid"].values
grid_all.append(midnight_grid_all)

# 조건을 or로 만족하는 그리드 번호 추출
grid_or = []
morning_grid_or = (morning_df[((morning_df["CCTV_num"] <= CCTV_num_mean) & (morning_df["police_value"] <= police_value_mean)) | ((morning_df["resident"] >= resident_mean) & (morning_df["floating"] >= floating_mean[0]))])["grid"].values
grid_or.append(morning_grid_or)

afternoon_grid_or = (afternoon_df[((afternoon_df["CCTV_num"] <= CCTV_num_mean) & (afternoon_df["police_value"] <= police_value_mean)) | ((afternoon_df["resident"] >= resident_mean) & (afternoon_df["floating"] >= floating_mean[1]))])["grid"].values
grid_or.append(afternoon_grid_or)

evening_grid_or = (evening_df[((evening_df["CCTV_num"] <= CCTV_num_mean) & (evening_df["police_value"] <= police_value_mean)) | ((evening_df["resident"] >= resident_mean) & (evening_df["floating"] >= floating_mean[2]))])["grid"].values
grid_or.append(evening_grid_or)

midnight_grid_or = (midnight_df[((midnight_df["CCTV_num"] <= CCTV_num_mean) & (midnight_df["police_value"] <= police_value_mean)) | ((midnight_df["resident"] >= resident_mean) & (midnight_df["floating"] >= floating_mean[3]))])["grid"].values
grid_or.append(midnight_grid_or)

for i in range(0, 4):
    for x in grid_or[i]:
        if x in grid_all[i]:
            grid_or[i] = np.delete(grid_or[i], np.where(grid_or[i] == x))